In [ ]:
%matplotlib inline
import time
import glob
import os
import cv2
import numpy as np
import pandas as pd
import h5py

import matplotlib.pyplot as plt

In [ ]:
# 데이터 학습시킬 때에는 256 x 256 사이즈를 이용할 것임
# 하지만 resize나 rotating할 때를 대비하여, padding을 추가하여
# 데이터셋을 구성할 예정
input_size = (384, 384) # 256 x 256 이미지에 0.5배의 padding을 추가시킴

image_dir = "../data/images"
profile_dir = "../data/profiles"
h5_path = "../data/baidu_segmentation.h5" # dataset을 hdf5 포맷으로 변경하여 저장
df = pd.read_csv("../data/person_count.csv") # 이미지에 몇 명의 사람 수가 있는지에 대한 자료

In [ ]:
# 데이터셋은 영상에 한명만 있는 경우만 포함함
filename_series = df[df.num_person == 1].filename
# image를 담을 데이터 셋
# dataset-shape : the number of data, height, width, channel
dataset = np.zeros((len(filename_series), *input_size, 4),dtype=np.uint8) 

start_time = time.time()
for idx, filename in enumerate(filename_series):
    # image와 profile 가져오기
    image_path = os.path.join(image_dir,filename)
    profile_path = os.path.join(profile_dir,filename)
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    profile = cv2.imread(profile_path, 0)
    # profile을 Color channel 중 4번째에 추가
    img_and_profile = np.concatenate([img,np.expand_dims(profile,axis=-1)],axis=2)
    # Padding을 추가시켜 정방행렬화
    h, w, _ = img_and_profile.shape
    if h - w > 0:
        diff = h - w
        pad1, pad2 = diff // 2 , diff - diff//2
        pad_input = np.pad(img_and_profile,((0,0),(pad1,pad2),(0,0)),'constant',constant_values=255)
    elif h - w < 0:
        diff = w - h
        pad1, pad2 = diff // 2 , diff - diff//2    
        pad_input = np.pad(img_and_profile,((pad1,pad2),(0,0),(0,0)),'constant',constant_values=255)
    else:
        pad_input = img_and_profile
    # Resize함
    resized = cv2.resize(pad_input, input_size)
    # dataset에 담음
    dataset[idx] = resized
    if idx % 100 == 0:
        print(idx,"th completed --- time : {}".format(time.time()-start_time))

# dataset 저장하기
with h5py.File(h5_path) as file:
    file.create_dataset("{}x{}".format(*input_size),
                        data=dataset,dtype=np.uint8)
print("save dataset in {}".format(h5_path))